Check connection to cluster is healthy

In [1]:
%status

{'status': 'healthy',
 'startTime': 'Sat May 20 04:26:30 UTC 2023',
 'dbEngineVersion': '1.2.0.2.R3',
 'role': 'writer',
 'dfeQueryEngine': 'viaQueryHint',
 'gremlin': {'version': 'tinkerpop-3.5.5'},
 'sparql': {'version': 'sparql-1.1'},
 'opencypher': {'version': 'Neptune-9.0.20190305-1.0'},
 'labMode': {'ObjectIndex': 'disabled',
  'ReadWriteConflictDetection': 'enabled'},
 'features': {'ResultCache': {'status': 'disabled'},
  'IAMAuthentication': 'disabled',
  'Streams': 'disabled',
  'AuditLog': 'disabled'},
 'settings': {'clusterQueryTimeoutInMs': '120000'}}

Test if the dataset is empty or not

In [9]:
%%oc
MATCH (n)
RETURN COUNT(n)

Delete all nodes and relationships

In [3]:
%%oc
MATCH (n)
DETACH DELETE n

Load graph data from S3 bucket

Source: s3://offshore-graph-data
Load ARN: arn:aws:iam::925004103030:role/NeptuneLoadFromS3

In [8]:
%load

Button(description='Submit', style=ButtonStyle())

Output()

Check loading status (replace Load ID with current ID)

In [6]:
%load_status 38b999bc-c99c-48fd-b7e6-179673cf9d5a

{
  "status": "200 OK",
  "payload": {
    "feedCount": [
      {
        "LOAD_COMPLETED": 1
      }
    ],
    "overallStatus": {
      "fullUri": "s3://graph-import",
      "runNumber": 1,
      "retryNumber": 0,
      "status": "LOAD_COMPLETED",
      "totalTimeSpent": 2041,
      "startTime": 1683249579,
      "totalRecords": 21486375,
      "totalDuplicates": 0,
      "parsingErrors": 0,
      "datatypeMismatchErrors": 0,
      "insertErrors": 0
    }
  }
}


Execute aggreation query in denormalized graph

In [10]:
%%oc
MATCH (e:Entity) WHERE
EXISTS(e.service_provider) AND EXISTS(e.sourceID) AND EXISTS(e.valid_until)
WITH e.service_provider AS provider, COUNT(*) AS amount
RETURN min(amount), max(amount), avg(amount)

Execute update on denormalized graph (min redundancy)

In [15]:
%%oc
MATCH (e:Entity)
WHERE EXISTS(e.service_provider) AND EXISTS(e.sourceID) AND EXISTS(e.valid_until)
AND e.service_provider = 'Appleby'
SET e.valid_until = 'Appleby data is current through 2015'

Execute update on denormalized graph (avg redundancy)

In [14]:
%%oc
MATCH (e:Entity)
WHERE EXISTS(e.service_provider) AND EXISTS(e.sourceID) AND EXISTS(e.valid_until)
AND e.service_provider = 'Portcullis Trustnet'
SET e.valid_until = 'The Offshore Leaks data is current through 2011'

Execute update on denormalized graph (max redundancy)

In [13]:
%%oc
MATCH (e:Entity)
WHERE EXISTS(e.service_provider) AND EXISTS(e.sourceID) AND EXISTS(e.valid_until)
AND e.service_provider = 'Mossack Fonseca'
SET e.valid_until = 'The Panama Papers data is current through 2016'

Normalize graph: create new nodes for each equivalence class

In [16]:
%%oc
MATCH (e:Entity) WHERE
EXISTS(e.service_provider) AND EXISTS(e.sourceID) AND EXISTS(e.valid_until)
WITH DISTINCT e.service_provider AS provider, e.sourceID AS ids, e.valid_until AS valid
CREATE (p:Provider{service_provider: provider, sourceID: ids, valid_until: valid})

Normalize graph: create edges and remove properties related to equivalence class from original nodes

In [17]:
%%oc
MATCH (e:Entity),(p:Provider) WHERE
EXISTS(e.service_provider) AND EXISTS(e.sourceID) AND EXISTS(e.valid_until) AND
e.service_provider = p.service_provider
CREATE (e)<-[:PROVIDED_TO]-(p)

Remove properties related to equivalence class from original nodes

In [19]:
%%oc
MATCH (e:Entity)
REMOVE e.service_provider, e.sourceID, e.valid_until

Execute aggreation query on normalized graph

In [20]:
%%oc
MATCH (p:Provider) WHERE
EXISTS(p.service_provider) AND EXISTS(p.sourceID) AND EXISTS(p.valid_until)
WITH SIZE((p)--()) AS amount
RETURN min(amount), max(amount), avg(amount)

Execute update on normalized graph (min redundancy)

In [21]:
%%oc
MATCH (p:Provider)
WHERE EXISTS(p.service_provider) AND EXISTS(p.sourceID) AND EXISTS(p.valid_until)
AND p.service_provider = 'Appleby'
SET p.valid_until = 'Appleby data is current through 2015'

Execute update on normalized graph (avg redundancy)

In [22]:
%%oc
MATCH (p:Provider)
WHERE EXISTS(p.service_provider) AND EXISTS(p.sourceID) AND EXISTS(p.valid_until)
AND p.service_provider = 'Portcullis Trustnet'
SET p.valid_until = 'The Offshore Leaks data is current through 2011'

Execute update on normalized graph (max redundancy)

In [23]:
%%oc
MATCH (p:Provider)
WHERE EXISTS(p.service_provider) AND EXISTS(p.sourceID) AND EXISTS(p.valid_until)
AND p.service_provider = 'Mossack Fonseca'
SET p.valid_until = 'The Panama Papers data is current through 2016'